# Federal University of Ceará
# Teleinformatics Departament
# Graduate Program in Teleinformatics Engeneering
## TIP8419 - Tensor Algebra
## Homework 7 - Multidimensional Least-Squares Khatri-Rao Factorization (MLS-KRF)
### Report and Simulation results

- Ezequias Márcio - 497779

To run this notebook properly, it is necessary Python3 installed alongside with the packages listed below:

- `numpy 1.17.2`
- `scipy 1.4.1`
- `tdqm 4.36.1`
- `bokeh 1.3.4`

Make sure that the files `tensoralg.py` and `ta_simulations.py` are in the same directory as this notebook. In this files, it can be found the tensor algebra module functions and the code listings of the simulations.

In [1]:
# Importing the simulation module:
from ta_simulations import *
np.set_printoptions(4, linewidth=175)
output_notebook()
# Loading the .m file given as a python dictionary:
krf_data = loadmat('files/krf_matrix_3D.mat')

Loading BokehJS ...

### Problem 1

On practice 3 we implement the LS-KRF (Least Square Khatri-Rao Factorization) algorithm, now we will go to implement the MLS-KRF (Multidimensional Least Square Khatri-Rao Factorization) algorithm. Then, Let $\mathbf{X} \approx \mathbf{A}^{(1)} \diamond \mathbf{A}^{(2)} \diamond ... \diamond \mathbf{A}^{(N)} \in \mathbb{C}^{I_{1} I_{2} ... I_{N} \times R}$ be a matrix composed by Khatri-Rao product of $N$ matrices $\mathbf{A}^{(n)} \in \mathbb{C}^{I_{n} \times R}$ , with $n = 1, 2, . . . , N$. For $N = 3$ and $R$ and $I_{n}$ arbitrary implement the MLS-KRF for that estimate $\mathbf{A}^{(1)}, \mathbf{A}^{(2)}$ and $\mathbf{A}^{(3)}$ by solving the following problem.

\begin{equation}
    (\hat{\mathbf{A}}^{(1)}, \hat{\mathbf{A}}^{(2)}, \hat{\mathbf{A}}^{(3)}) = \underset{\mathbf{A}, \mathbf{B}}{\text{min}} || \mathbf{X} - \mathbf{A}^{(1)}       \diamond \mathbf{A}^{(2)} \diamond \mathbf{A}^{(3)} ||^{2}_{F}
\end{equation}

Compare the estimate matrices $\hat{\mathbf{A}}^{(1)}, \hat{\mathbf{A}}^{(2)}$ and $\hat{\mathbf{A}}^{(3)}$ with the original ones. What can you
conclude? Explain the results.

### Solution:

In the cell below, the matrices $\mathbf{A}^{(1)} \in \mathbb{C}^{4×2}$, $\mathbf{A}^{(2)} \in \mathbb{C}^{3×2}$ and $\mathbf{A}^{(3)} \in \mathbb{C}^{2×2}$ are randomly generated and is calculated the squared error between these matrices and their estimates for comparison. The implemented routine for this factorization is in the file `tensoralg.py` as `mlskrf`. Note that the functions implemented are generalized for the $N$ matrices case.

In [2]:
# Testing the Multidimensional Least-Squares Khatri-Rao Factorization:
rows_a = [3, 2, 3]; ncol = 2 # Number of rows and columns for each matrix A
A = [rand(row, ncol*2).view(np.complex_) for row in rows_a] # Generating random matrices: A1, A2, A3
X = tensoralg.kr(*A); print(f'Matrix X {X.shape}') # Construct matrix X
# Estimating matrices A_hat and X_hat:
A_hat = tensoralg.mlskrf(X, rows_a); X_hat = tensoralg.kr(*A_hat)
# Calculating the Normalized Mean Squared Error:
nmse_A = [norm_mse(A[n], A_hat[n]) for n in range(len(rows_a))]; nmse_X = norm_mse(X, X_hat)
print(f'''Normalized Mean Squared Errors:\n- Matrix A: {nmse_A}\n- Matrix X: {nmse_X}''')
print('\nUsing HOOI:')
# Estimating matrices A_hat and X_hat:
A_hat = tensoralg.mlskrf(X, rows_a, True); X_hat = tensoralg.kr(*A_hat)
# Calculating the Normalized Mean Squared Error:
nmse_A = [norm_mse(A[n], A_hat[n]) for n in range(len(rows_a))]; nmse_X = norm_mse(X, X_hat)
print(f'''Normalized Mean Squared Errors:\n- Matrix A: {nmse_A}\n- Matrix X: {nmse_X}''')

Matrix X (18, 2)
Normalized Mean Squared Errors:
- Matrix A: [2.2792483438662905, 5.068258653669103, 2.242032220146126]
- Matrix X: 3.2524865962529233e-31

Using HOOI:
Normalized Mean Squared Errors:
- Matrix A: [2.2792483438662905, 5.0682586536691, 2.242032220146125]
- Matrix X: 6.805071697259927e-31


As can be seen by the result above, the error between the estimated matrices $\hat{\mathbf{A}}^{(n)}$ and the oringinal ones is high. However, the error between the reconstructed version of $\mathbf{X}$ $(\hat{\mathbf{A}}^{(1)}\diamond \hat{\mathbf{A}}^{(2)} \diamond \hat{\mathbf{A}}^{(3)})$ and the original is minimized by the MLSKRF algorithm.

Again, as in the LSKRF case, because only the first $n$-th singular value and vector are chosen for the truncated HOSVD/HOOI, the estimated matrices are not close to the original ones, that is evidenced by the high error values above. Also, for the MLSKRF using the HOOI for the estimation the obtained result is according to the expected.

- Validating the results:

Using the provided matrices in `krf_matrix_3D.mat`, it can be seen below that the implemented algorithm is working as expected, presenting a small error for the reconstructed matrix.

In [3]:
# Validating the results with the matrices provided:
a_test = krf_data['A']; b_test = krf_data['B']; c_test = krf_data['C']; x_test = krf_data['X']
print(f'Matrix X {x_test.shape}\nMatrix A {a_test.shape}\nMatrix B {b_test.shape}\nMatrix C {c_test.shape}')
# Estimating matrices A, B and C:
matrices = [a_test, b_test, c_test]; test_rows = [m.shape[0]for m in matrices]
m_hat = tensoralg.mlskrf(x_test, test_rows); x_hat = tensoralg.kr(*m_hat)
# Calculating the squared error
# Calculating the Normalized Mean Squared Error:
nmse_test = [norm_mse(matrices[n], m_hat[n]) for n in range(len(test_rows))]; nmse_x_test = norm_mse(x_test, x_hat)
print(f'''Normalized Mean Squared Errors:\n- Matrices A, B, C: {nmse_test}\n- Matrix X: {nmse_x_test}''')

Matrix X (160, 4)
Matrix A (5, 4)
Matrix B (4, 4)
Matrix C (8, 4)
Normalized Mean Squared Errors:
- Matrices A, B, C: [1.6102677822647011, 2.6779127326052063, 2.7774333262997164]
- Matrix X: 1.796797491781918e-31


### Problem 2: 

Assuming 1000 Monte Carlo experiments, generate  $\mathbf{X_{0}} = \mathbf{A}\diamond \mathbf{B} \diamond \mathbf{C} \in \mathbb{C}^{I_{1} I_{2} I_{3} \times R}$, for randomly chosen $\mathbf{A} \in \mathbb{C}^{I_{1} \times R}$, $\mathbf{B} \in \mathbb{C}^{I_{2} \times R}$ and $\mathbf{C} \in \mathbb{C}^{I_{3} \times R}$, with $R = 4$, whose elements are drawn from a normal distribution. Let $\mathbf{X} = \mathbf{X_{0}} + \alpha \mathbf{V}$ be a noisy version of $\mathbf{X}$ where $\mathbf{V}$ is the additive noise term, whose elements are drawn from a normal distribution. The parameter $\alpha$ controls the power (variance) of the noise term, and is defined as a function of the signal to noise ratio (SNR), in dB, as follows:

\begin{equation}
    \text{SNR}_{dB} = 10 \log_{10} \frac{||\mathbf{X_{0}}||^{2}_{F}}{||\alpha \mathbf{V}||^{2}_{F}}
\end{equation}

Assuming the SNR range $[0, 5, 10, 15, 20, 25, 30]$ dB, find the estimates $\hat{\mathbf{A}}$, $\hat{\mathbf{B}}$ and $\hat{\mathbf{C}}$ obtained with the MLS-KRF algorithm for the configurations $I_{1} = 2$, $I_{2} = 3$ and $I_{3} = 4$. 

Let us define the normalized mean square error (NMSE) measure as follows:

\begin{equation}
    \text{NMSE}(\mathbf{X}_{0}) = \frac{1}{1000} \sum^{1000}_{i = 1} \frac{||\hat{\mathbf{X}}_{0}(i) - \mathbf{X}_{0}(i)||^{2}_{F}}{||\mathbf{X}_{0}(i)||^{2}_{F}}\end{equation}  

where $\mathbf{X}_{0}(i)$ and $\hat{\mathbf{X}}_{0}(i)$ represent the original data matrix and the reconstructed one at the ith experiment, respectively. For each SNR value and configuration, plot the NMSE vs. SNR curve. Discuss the obtained results.

### Solution:

The simulation results are generated in the cell below. First, for the 1000 realizations, the matrix $\mathbf{X}_{0}(i)$ is generated and then, for each value of SNR, it is added Gaussian noise to the matrix tha will be the input of the LSKRF algorithm. After that, the estimated matrices $\hat{\mathbf{A}}, \hat{\mathbf{B}}$ and $\hat{\mathbf{C}}$ are used to build the estimative $\hat{\mathbf{X}}_{0}(i)$. Lastly, the NMSE between the estimated matrix and the original is sorted. This process is implemented in the function `run_simulation_mlskr`.

In [4]:
ncol = 4; nrows = [2, 3, 4]  # Number of rows I1, I2 and I3 and Number of columns R
snr = np.linspace(0, 30, 7)  # SNR values
mc_realizations = 1000       # Monte Carlo Realizations
#Generating data for the two cases: HOSVD and HOOI:
result1 = run_simulation_mlskr(snr, mc_realizations, nrows, ncol)
result2 = run_simulation_mlskr(snr, mc_realizations, nrows, ncol, True)

The NMSE vs. SNR curve is presented in a log-scale plot below.

As can be seen in the figure below, with the increase in SNR, the estimative is more accurate because the noise power is decreasing. Also, with respect to the case where the HOOI algorithm is applied the performance of the MLSKRF algorithm is practically the same.

In [5]:
plot_results(snr, result1, result2, 'using HOSVD', 'using HOOI', 'MLSKRF')